In [6]:
import sgkit as sg
import pandas as pd
import numpy as np
import zarr
import xarray as xr
import allel

In [7]:
import sgkit as sg

ds = sg.load_dataset('/Users/christinagkertsou/Desktop/Finalyearproject/dataset/AgamP4_2L.zarr')
ds

ds2 = sg.load_dataset('/Users/christinagkertsou/Desktop/Finalyearproject/dataset/AgamP4_2R.zarr')
ds2

ds3 = sg.load_dataset('/Users/christinagkertsou/Desktop/Finalyearproject/dataset/AgamP4_3L.zarr')
ds3 

ds4 = sg.load_dataset('/Users/christinagkertsou/Desktop/Finalyearproject/dataset/AgamP4_3R.zarr')
ds4 

ds5 = sg.load_dataset('/Users/christinagkertsou/Desktop/Finalyearproject/dataset/AgamP4_X.zarr')
ds5 

<xarray.Dataset>
Dimensions:               (variants: 684159, samples: 18, alleles: 4,
                           ploidy: 2, alt_alleles: 3)
Dimensions without coordinates: variants, samples, alleles, ploidy, alt_alleles
Data variables: (12/15)
    call_AD               (variants, samples, alleles) int32 dask.array<chunksize=(10000, 18, 4), meta=np.ndarray>
    call_DP               (variants, samples) int32 dask.array<chunksize=(10000, 18), meta=np.ndarray>
    call_GQ               (variants, samples) int32 dask.array<chunksize=(10000, 18), meta=np.ndarray>
    call_genotype         (variants, samples, ploidy) int8 dask.array<chunksize=(10000, 18, 2), meta=np.ndarray>
    call_genotype_mask    (variants, samples, ploidy) bool dask.array<chunksize=(10000, 18, 2), meta=np.ndarray>
    call_genotype_phased  (variants, samples) bool dask.array<chunksize=(10000, 18), meta=np.ndarray>
    ...                    ...
    variant_MQ            (variants) float32 dask.array<chunksize=(10000,), meta=np.ndarray>
    variant_allele        (variants, alleles) object dask.array<chunksize=(10000, 4), meta=np.ndarray>
    variant_contig        (variants) int8 dask.array<chunksize=(10000,), meta=np.ndarray>
    variant_id            (variants) object dask.array<chunksize=(10000,), meta=np.ndarray>
    variant_id_mask       (variants) bool dask.array<chunksize=(10000,), meta=np.ndarray>
    variant_position      (variants) int32 dask.array<chunksize=(10000,), meta=np.ndarray>
Attributes:
    contigs:               ['AgamP4_2L', 'AgamP4_2R', 'AgamP4_3L', 'AgamP4_3R...
    max_alt_alleles_seen:  4
    source:                sgkit-0.4.0

In [8]:
    ds['sample_cohort'] = xr.DataArray([0] * 2 + [1] * 10 + [2] * 6, dims="samples")
    ds['sample_cohort_name'] = xr.DataArray(['ara'] * 2 + ['gam'] * 10 + ['hyb_ctr'] + ['hyb'] * 5, dims="samples")
    ds['sample_cohort_name']

<xarray.DataArray 'sample_cohort_name' (samples: 18)>
array(['ara', 'ara', 'gam', 'gam', 'gam', 'gam', 'gam', 'gam', 'gam',
       'gam', 'gam', 'gam', 'hyb_ctr', 'hyb', 'hyb', 'hyb', 'hyb', 'hyb'],
      dtype='<U7')
Dimensions without coordinates: samples

In [9]:
sample_is_hyb = ds['sample_cohort_name'] == 'hyb'
sample_is_ara = ds['sample_cohort_name'] == 'ara'
sample_is_gam = ds['sample_cohort_name'] == 'gam'

In [10]:
g = allel.GenotypeArray(ds['call_genotype'])

In [6]:
#ara_no_call

is_ara_missing=g[:,sample_is_ara].is_missing().all(axis=1)


In [8]:
is_fixed_hyb = g[:,sample_is_hyb].is_hom_alt().sum(axis=1) >= 3
is_fixed_hyb

array([False, False, False, ..., False, False, False])

In [13]:
is_ara_hom_ref=g[:,sample_is_ara].is_hom_ref().all(axis=1)
g_markers_hom = g[is_ara_hom_ref,:]

In [15]:
is_ara_hom_ref=g[:,sample_is_ara].is_hom_ref().all(axis=1)
g_markers_hom_ref = g[is_ara_hom_ref,:]

In [18]:
is_ara_hom_alt=g[:,sample_is_ara].is_hom_alt().all(axis=1)
g_markers_hom_alt = g[is_ara_hom_alt,:]

In [19]:
is_ara_missing=g[:,sample_is_ara].is_missing().all(axis=1)
is_ara_hom_ref=g[:,sample_is_ara].is_hom_ref().all(axis=1)
is_ara_hom_alt=g[:,sample_is_ara].is_hom_alt().all(axis=1)


In [20]:
is_hyb_hom_alt = g[:,sample_is_hyb].is_hom_alt().sum(axis=1) >= 3
is_hyb_missing=g[:,sample_is_hyb].is_missing().sum(axis=1) >= 3
is_hyb_hom_ref=g[:,sample_is_hyb].is_hom_ref().sum(axis=1) >= 3

In [21]:
mask_ara_no_hyb_alt=is_hyb_hom_alt & is_ara_missing
sum(mask_ara_no_hyb_alt)

4497

In [22]:
mask_ara_ref_hyb_alt=is_hyb_hom_alt & is_ara_hom_ref
sum(mask_ara_ref_hyb_alt)

0

In [23]:
mask_ara_alt_hyb_no=is_hyb_missing & is_ara_hom_alt
sum(mask_ara_alt_hyb_no)

924735

In [24]:
mask_ara_alt_hyb_ref=is_hyb_hom_ref & is_ara_hom_alt
sum(mask_ara_alt_hyb_ref)

0

In [27]:
mask_fin= mask_ara_no_hyb_alt | mask_ara_ref_hyb_alt | mask_ara_alt_hyb_no | mask_ara_alt_hyb_ref
mask_fin
sum(mask_fin)

929232

In [28]:
929232/1855171

0.5008875192637229

In [37]:
mask_final = np.array([mask_ara_no_hyb_alt, mask_ara_ref_hyb_alt, mask_ara_alt_hyb_no, mask_ara_alt_hyb_ref]).any(axis=0)

In [39]:
mask_final.sum()

929232

In [2]:
def masks_ara_hyb(ds):

    ds['sample_cohort'] = xr.DataArray([0] * 2 + [1] * 10 + [2] * 6, dims="samples")
    ds['sample_cohort_name'] = xr.DataArray(['ara'] * 2 + ['gam'] * 10 + ['hyb_ctr'] + ['hyb'] * 5, dims="samples")
    ds['sample_cohort_name']

    sample_is_hyb = ds['sample_cohort_name'] == 'hyb'
    sample_is_ara = ds['sample_cohort_name'] == 'ara'
    sample_is_gam = ds['sample_cohort_name'] == 'gam'
    
    g = allel.GenotypeArray(ds['call_genotype'])

    is_ara_missing=g[:,sample_is_ara].is_missing().all(axis=1)
    is_ara_hom_ref=g[:,sample_is_ara].is_hom_ref().all(axis=1)
    is_ara_hom_alt=g[:,sample_is_ara].is_hom_alt().all(axis=1)

    is_hyb_hom_alt = g[:,sample_is_hyb].is_hom_alt().sum(axis=1) >= 3
    is_hyb_missing=g[:,sample_is_hyb].is_missing().sum(axis=1) >= 3
    is_hyb_hom_ref=g[:,sample_is_hyb].is_hom_ref().sum(axis=1) >= 3

    mask_ara_no_hyb_alt=is_hyb_hom_alt & is_ara_missing

    mask_ara_ref_hyb_alt=is_hyb_hom_alt & is_ara_hom_ref

    mask_ara_alt_hyb_no=is_hyb_missing & is_ara_hom_alt

    mask_ara_alt_hyb_ref=is_hyb_hom_ref & is_ara_hom_alt

    mask_fin= mask_ara_no_hyb_alt | mask_ara_ref_hyb_alt | mask_ara_alt_hyb_no | mask_ara_alt_hyb_ref

    return mask_fin.sum()
        
    


In [3]:
num_snps_new= {}

for contig in ['AgamP4_2L', 'AgamP4_2R', 'AgamP4_3L', 'AgamP4_3R', 'AgamP4_X']:
    print(contig)
    ds = sg.load_dataset(f'/Users/christinagkertsou/Desktop/Finalyearproject/dataset/{contig}.zarr')
    num_snps_new[contig] = []
    n = masks_ara_hyb(ds)
    print(n)


AgamP4_2L
929232
AgamP4_2R
742087
AgamP4_3L
387033
AgamP4_3R
630705
AgamP4_X
453185


In [32]:
def masks_ara_hyb(ds, thres_number):

    ds['sample_cohort'] = xr.DataArray([0] * 2 + [1] * 10 + [2] * 6, dims="samples")
    ds['sample_cohort_name'] = xr.DataArray(['ara'] * 2 + ['gam'] * 10 + ['hyb_ctr'] + ['hyb'] * 5, dims="samples")
    ds['sample_cohort_name']

    sample_is_hyb = ds['sample_cohort_name'] == 'hyb'
    sample_is_ara = ds['sample_cohort_name'] == 'ara'
    sample_is_gam = ds['sample_cohort_name'] == 'gam'
    
    g = allel.GenotypeArray(ds['call_genotype'])

    is_ara_missing=g[:,sample_is_ara].is_missing().all(axis=1)
    is_ara_hom_ref=g[:,sample_is_ara].is_hom_ref().all(axis=1)
    is_ara_hom_alt=g[:,sample_is_ara].is_hom_alt().all(axis=1)

    is_hyb_hom_alt = g[:,sample_is_hyb].is_hom_alt().sum(axis=1) >= thres_number
    is_hyb_missing=g[:,sample_is_hyb].is_missing().sum(axis=1) >= thres_number
    is_hyb_hom_ref=g[:,sample_is_hyb].is_hom_ref().sum(axis=1) >= thres_number

    mask_ara_no_hyb_alt=is_hyb_hom_alt & is_ara_missing

    mask_ara_ref_hyb_alt=is_hyb_hom_alt & is_ara_hom_ref

    mask_ara_alt_hyb_no=is_hyb_missing & is_ara_hom_alt

    mask_ara_alt_hyb_ref=is_hyb_hom_ref & is_ara_hom_alt

    mask_fin= mask_ara_no_hyb_alt | mask_ara_ref_hyb_alt | mask_ara_alt_hyb_no | mask_ara_alt_hyb_ref

    # return mask_fin.sum()
    return mask_fin

In [18]:
num_snps_new= {}

for contig in ['AgamP4_2L', 'AgamP4_2R', 'AgamP4_3L', 'AgamP4_3R', 'AgamP4_X']:
    print(contig)
    ds = sg.load_dataset(f'/Users/christinagkertsou/Desktop/Finalyearproject/dataset/{contig}.zarr')
    num_snps_new[contig] = []
   

    for threshold in [2, 3, 4, 5]:
        print(threshold,)
        n = masks_ara_hyb(ds, threshold)
        num_snps_new[contig].append(n)
        print(n)

AgamP4_2L
2
934633
3
929232
4
923519
5
911300
AgamP4_2R
2
747219
3
742087
4
736926
5
726048
AgamP4_3L
2
389881
3
387033
4
384154
5
378303
AgamP4_3R
2
633911
3
630705
4
627151
5
619259
AgamP4_X
2
455727
3
453185
4
450633
5
445824


In [22]:
import pandas as pd

snp_count_thresh_df = pd.DataFrame(num_snps_new).T
snp_count_thresh_df.columns = ['thr_2', "thr_3",'thr_4', "thr_5" ]
snp_count_thresh_df

snp_count_thresh_df.to_csv("Snp_count_thresh.csv")

In [22]:
import pyranges as pr

# import annotations
ann_db = pr.read_gff3("/Users/christinagkertsou/Desktop/VectorBase-56_AgambiaePEST.gff")
ann_db = ann_db['AgamP4_2L']
ann_db

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,ID,description,Parent,gene_id,protein_source_id,Name,Note
0,AgamP4_2L,VEuPathDB,protein_coding_gene,38696490,38697681,.,+,.,AGAP006810,chromatin modification-related protein EAF6,NaN,NaN,NaN,NaN,NaN
1,AgamP4_2L,VEuPathDB,mRNA,38696490,38697681,.,+,.,AGAP006810-RA,chromatin modification-related protein EAF6,AGAP006810,NaN,NaN,NaN,NaN
2,AgamP4_2L,VEuPathDB,exon,38696490,38696664,.,+,.,exon_AGAP006810-RA-E1,NaN,AGAP006810-RA,AGAP006810,NaN,NaN,NaN
3,AgamP4_2L,VEuPathDB,exon,38696790,38696994,.,+,.,exon_AGAP006810-RA-E2,NaN,AGAP006810-RA,AGAP006810,NaN,NaN,NaN
4,AgamP4_2L,VEuPathDB,exon,38697074,38697681,.,+,.,exon_AGAP006810-RA-E3,NaN,AGAP006810-RA,AGAP006810,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43788,AgamP4_2L,VEuPathDB,exon,31566897,31566994,.,-,.,exon_AGAP006408-RA-E1,NaN,AGAP006408-RA,AGAP006408,NaN,NaN,NaN
43789,AgamP4_2L,VEuPathDB,CDS,31565063,31566794,.,-,0,AGAP006408-PA-CDS1,NaN,AGAP006408-RA,AGAP006408,AGAP006408-PA,NaN,NaN
43790,AgamP4_2L,VEuPathDB,three_prime_UTR,31564827,31565063,.,-,.,utr_AGAP006408-RA_1,NaN,AGAP006408-RA,NaN,NaN,NaN,NaN
43791,AgamP4_2L,VEuPathDB,five_prime_UTR,31566794,31566825,.,-,.,utr_AGAP006408-RA_2,NaN,AGAP006408-RA,NaN,NaN,NaN,NaN


In [35]:
#to fix

# original array is filtered to contain only ARA samples (edo)
g_ara = g[:,sample_is_ara,:]

# we make a mask to select only variants that are called in both ara samples
mask_g_ara_is_called = g_ara.is_called().all(axis=1)

# we apply mask to ARA samples to keep only variants called in both samples
g_ara_called = g_ara[mask_g_ara_is_called,:,:]
g_ara_called

# original array is filtered to contain only ARA samples (edo)
g_hyb = g[:,sample_is_hyb,:]

# we make a mask to select only variants that are called in both ara samples
mask_g_hyb_is_called = g_hyb.is_called().all(axis=1)

# we apply mask to ARA samples to keep only variants called in both samples
g_hyb_called = g_hyb[mask_g_hyb_is_called,:,:]
g_hyb_called




<GenotypeArray shape=(18660, 5, 2) dtype=int8>
1/1 1/1 1/1 1/1 1/1
1/1 1/1 1/1 1/1 1/1
1/1 1/1 1/1 1/1 1/1
...
0/1 0/1 1/1 0/1 1/1
0/1 0/1 0/1 0/1 0/1
0/1 0/1 0/1 0/1 0/1

In [23]:
g = allel.GenotypeArray(ds['call_genotype'])

mask_g_ara_is_called=g[:,sample_is_ara].is_called().all(axis=1)



In [24]:
mask_g_hyb_is_called = g[:,sample_is_hyb,:].is_called().all(axis=1)


In [25]:
mask_ara_hyb_pos = mask_g_ara_is_called | mask_g_hyb_is_called
mask_ara_hyb_pos

array([False, False, False, ...,  True,  True,  True])

In [26]:
pos = allel.SortedIndex(ds['variant_position'])

pos_masked = pos[mask_ara_hyb_pos]
pos_masked

0,1,2,3,4,...,1235301,1235302,1235303,1235304,1235305
3894,3913,4235,4260,7580,...,49359034,49359061,49359453,49359464,49359493


In [28]:
import pyranges as pr

markers = pr.from_dict([{'Chromosome': 'AgamP4_2L', 'Start': p, 'End': p} for p in pos_masked])
markers

,Chromosome,Start,End
0,AgamP4_2L,3894,3894
1,AgamP4_2L,3913,3913
2,AgamP4_2L,4235,4235
3,AgamP4_2L,4260,4260
4,AgamP4_2L,7580,7580
...,...,...,...
1235301,AgamP4_2L,49359034,49359034
1235302,AgamP4_2L,49359061,49359061
1235303,AgamP4_2L,49359453,49359453
1235304,AgamP4_2L,49359464,49359464


In [29]:
count_df = pr.count_overlaps({'markers': markers}, features=ann_db).df

In [30]:
count_df.sort_values('markers', ascending=False).to_csv('counts.tsv', sep='\t')

In [33]:
het_mask = masks_ara_hyb(ds, 3)

In [35]:
het_mask

929232

In [36]:
g[het_mask,:,:]

<GenotypeArray shape=(929232, 18, 2) dtype=int8>
1/1 1/1 ./. ./. ./. ... ./. ./. ./. ./. ./.
1/1 1/1 ./. ./. ./. ... ./. ./. ./. ./. ./.
1/1 1/1 ./. ./. ./. ... ./. ./. ./. ./. ./.
...
1/1 1/1 ./. ./. ./. ... ./. ./. ./. ./. ./.
1/1 1/1 ./. ./. ./. ... ./. ./. ./. ./. ./.
1/1 1/1 ./. ./. ./. ... ./. ./. ./. ./. ./.

In [40]:
pos_slected_hets = pos[het_mask]
pos_slected_hets

0,1,2,3,4,...,929227,929228,929229,929230,929231
9626,9644,9655,10583,10590,...,49358995,49359061,49359453,49359464,49359493


In [ ]:
markers_sel_hets = pr.from_dict([{'Chromosome': 'AgamP4_2L', 'Start': p, 'End': p} for p in pos_slected_hets])
